In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path 
#from rich import inspect

In [125]:
## Build'n'Load Stuff (paths and arrays)
freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data")
sessionID = 'session_SER_1_20_22'
this_freemocap_session_path = freemocap_validation_data_path / sessionID
this_freemocap_data_path = this_freemocap_session_path/'DataArrays'
mediapipe_data_path = this_freemocap_data_path / 'mediaPipeSkel_3d_smoothed.npy'


In [126]:

mediapipeSkel_fr_mar_dim = np.load(mediapipe_data_path)

In [127]:
num_pose_joints = 33
first_frame = 0
last_frame = 15000

pose_joint_range = range(num_pose_joints)
frame_range = range(first_frame,last_frame)

mediapipe_pose_data = mediapipeSkel_fr_mar_dim[frame_range[0]:frame_range[-1],pose_joint_range[0]:pose_joint_range[-1],:]
skel_x = mediapipe_pose_data[:,:,0]
skel_y = mediapipe_pose_data[:,:,1]
skel_z = mediapipe_pose_data[:,:,2]

In [128]:
good_frame = 5000
goodframe_x = skel_x[good_frame,:]
goodframe_y = skel_y[good_frame,:]
goodframe_z = skel_z[good_frame,:]
# mediapipe_good_frame_data = mediaPipeSkel_fr_mar_dim[good_frame,:,:]
# skel_x = mediapipe_good_frame_data[:,0]
# skel_y = mediapipe_good_frame_data[:,1]
# skel_z = mediapipe_good_frame_data[:,2]

In [129]:
def build_mediapipe_skeleton(mediapipe_pose_data):

    r_shoulder_mp_index = 11
    l_shoulder_mp_index = 12
    r_elbow_mp_index = 13
    l_elbow_mp_index = 14
    r_wrist_mp_index = 15
    l_wrist_mp_index = 16
    r_hip_mp_index = 23
    l_hip_mp_index = 24
    r_knee_mp_index = 25
    l_knee_mp_index = 26
    r_ankle_mp_index = 27
    l_ankle_mp_index = 28
    r_heel_mp_index = 29
    l_heel_mp_index = 30
    r_foot_mp_index = 31
    l_foot_mp_index = 32

    #mediapipe_pose_skeleton_connections = {'r_upper_arm':{'proximal':r_shoulder_mp_index,'distal':r_elbow_mp_index},'l_upper_arm':{'proximal':l_shoulder_mp_index,'distal':l_elbow_mp_index}}
    mediapipe_pose_skeleton_connections = {'r_upper_arm':[r_shoulder_mp_index,r_elbow_mp_index],'l_upper_arm':[l_shoulder_mp_index,l_elbow_mp_index],'r_forearm':[r_elbow_mp_index,r_wrist_mp_index], 'l_forearm':[l_elbow_mp_index,l_wrist_mp_index], 'r_thigh':[r_hip_mp_index,r_knee_mp_index],'l_thigh':[l_hip_mp_index,l_knee_mp_index],
    'r_shank':[r_knee_mp_index,r_ankle_mp_index],'l_shank':[l_knee_mp_index,l_ankle_mp_index]}
    mediapipe_pose_skeleton_coordinates = {}
  
    for body_segment in mediapipe_pose_skeleton_connections.keys():
        proximal_joint_index = mediapipe_pose_skeleton_connections[body_segment][0]
        distal_joint_index = mediapipe_pose_skeleton_connections[body_segment][1]
        mediapipe_pose_skeleton_coordinates[body_segment] = {'proximal':mediapipe_pose_data[:,proximal_joint_index,:],'distal':mediapipe_pose_data[:,distal_joint_index,:]}

    return mediapipe_pose_skeleton_coordinates
    f= 2
skelcoordinates_segment_joint_frames_XYZ = build_mediapipe_skeleton(mediapipe_pose_data)

f=2 

In [130]:
def calculate_segment_COM(frame,skelcoordinates_segment_joint_frames_XYZ):
    
    COM_segment_lengths = {'r_upper_arm':.5772,'l_upper_arm':.5772,'l_forearm':.4574, 'r_forearm':.4574,'r_thigh':.4095,'l_thigh':.4095,'r_shank':.4459,'l_shank':.4459}
    coordinate_dict = {'x':0,'y':1,'z':2}
    coordinate_COM = {'x':None,'y':None,'z':None}
    segment_COM_dict = {}
    segment_COM_x = []
    segment_COM_y = []
    segment_COM_z = []
    for body_segment in skelcoordinates_segment_joint_frames_XYZ.keys():
        this_segment = skelcoordinates_segment_joint_frames_XYZ[body_segment]
        this_segment_proximal = this_segment['proximal']
        this_segment_distal = this_segment['distal']
        thisframe_coordinate_COM = coordinate_COM.copy()
        for coordinate in coordinate_dict.keys():
            coordinate_value = coordinate_dict[coordinate]
            thisframe_coordinate_COM[coordinate] = this_segment_proximal[frame][coordinate_value] + COM_segment_lengths[body_segment]*(this_segment_distal[frame][coordinate_value]-this_segment_proximal[frame][coordinate_value])
            f =2 
        segment_COM_dict[body_segment]= thisframe_coordinate_COM
        
    return segment_COM_dict

def reformat_segment_COM(segment_COM_dict):
    segment_COM_x = []
    segment_COM_y = []
    segment_COM_z = []
    
    for body_segment in segment_COM_dict:
        segment_COM_x.append(segment_COM_dict[body_segment]['x'])
        segment_COM_y.append(segment_COM_dict[body_segment]['y'])
        segment_COM_z.append(segment_COM_dict[body_segment]['z'])

    return segment_COM_x,segment_COM_y,segment_COM_z



segment_COM_dict = calculate_segment_COM(good_frame,skelcoordinates_segment_joint_frames_XYZ)
segment_COM_x,segment_COM_y,segment_COM_z = reformat_segment_COM(segment_COM_dict)
f = 2

In [131]:
%matplotlib
plt.ion()
figure = plt.figure()
ax = figure.add_subplot(111, projection = '3d')
ax.scatter(goodframe_x, goodframe_y,goodframe_z)
ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z)
plt.show()
# inspect(ax,methods= True)

Using matplotlib backend: TkAgg


In [25]:
# #http://www.kdm.p.lodz.pl/articles/2017/3/21_3_4.pdf
# r_upperarm_COM_x = goodframe_x[r_upperarm[0]] + (.5772)*(goodframe_x[r_upperarm[1]]-goodframe_x[r_upperarm[0]])
# l_upperarm_COM_x = goodframe_x[l_upperarm[0]] + (.5772)*(goodframe_x[l_upperarm[1]]-goodframe_x[l_upperarm[0]])

# r_forearm_COM_x = goodframe_x[r_forearm[0]] + (.4574)*(goodframe_x[r_forearm[1]]-goodframe_x[r_forearm[0]])
# l_forearm_COM_x = goodframe_x[l_forearm[0]] + (.4574)*(goodframe_x[l_forearm[1]]-goodframe_x[l_forearm[0]])

# r_upperarm_COM_y = goodframe_y[r_upperarm[0]] + (.5772)*(goodframe_y[r_upperarm[1]]-goodframe_y[r_upperarm[0]])
# l_upperarm_COM_y = goodframe_y[l_upperarm[0]] + (.5772)*(goodframe_y[l_upperarm[1]]-goodframe_y[l_upperarm[0]])

# r_forearm_COM_y = goodframe_y[r_forearm[0]] + (.4574)*(goodframe_y[r_forearm[1]]-goodframe_y[r_forearm[0]])
# l_forearm_COM_y = goodframe_y[l_forearm[0]] + (.4574)*(goodframe_y[l_forearm[1]]-goodframe_y[l_forearm[0]])

# segment_com_x = [r_upperarm_COM_x,l_upperarm_COM_x,r_forearm_COM_x,l_forearm_COM_x]
# segment_com_y = [r_upperarm_COM_y,l_upperarm_COM_y,r_forearm_COM_y,l_forearm_COM_y]



In [6]:
# %matplotlib
# plt.ion()

In [8]:
# ax.scatter(goodframe_x, goodframe_y, goodframe_z)
# plt.show()

In [15]:
# #x_com = x_proximal + (length_per)*(x_distal - x_proximal)

# #arms
# r_upperarm = [11,13]
# r_forearm = [13,15]

# l_upperarm = [12,14]
# l_forearm = [14,16]

# #legs
# r_thigh = [23,25]
# r_lowerleg = [25,27]
# r_foot = [29,31]

# l_thigh = [24,26]
# l_lowerleg = [26,28]
# l_foot = [30,32]


In [ ]:
#x_com = r_upperarm_COM_x*.00325 + l_upperarm_COM_x*.00325 + r_forearm_COM_x*